In [2]:
import pandas as pd
import numpy as np

In [3]:
dfMeta = pd.read_csv('movies_metadata_clean.csv')
dfRate = pd.read_csv('movies_ratings_small.csv')

In [4]:
print(dfMeta.columns)
print(dfRate.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


In [5]:
dfMeta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,10/30/1995,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,12/15/1995,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,12/22/1995,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,12/22/1995,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,2/10/1995,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173


In [6]:
dfRate.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


# Want to try to create a recommendation based on user ratings to pair movies using SVD method

Based on method found in multiple sources from data scientists with a similar dataset

In [10]:
# Format matrix similar to retail baskets
dfMat = dfRate.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
dfMat.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Convert to numpy array
R = dfMat.to_numpy()

# Normalize by subtracting the mean user rating
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [12]:
from sklearn.decomposition import TruncatedSVD

mdlSvd = TruncatedSVD(n_components=50, algorithm='arpack', random_state=1)
pd.DataFrame(mdlSvd.fit_transform(R_demeaned) + user_ratings_mean.reshape(-1, 1))

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.906105,0.939455,0.334835,-0.731930,-0.366518,0.234075,0.568111,-0.204842,-0.608238,0.162166,...,0.409540,0.292093,0.496599,-0.199819,0.016697,-0.309424,-0.311894,0.084273,-0.172555,0.016972
1,9.445424,2.682744,-13.221284,11.363585,0.089489,1.916099,2.423597,1.521079,-2.043288,-1.274034,...,0.074771,0.818089,-0.727269,0.433260,0.391865,0.835994,-0.442794,-0.269483,-1.257899,0.836545
2,7.984568,-2.653126,-1.568858,2.893915,-0.671719,-2.005335,-2.216680,1.682211,2.815122,-0.331325,...,-0.592482,-0.762039,-0.216471,-0.085983,-1.354265,-0.888138,-1.173256,-1.194269,0.620423,2.254003
3,26.911493,10.216111,-6.747388,-14.657223,-5.824021,-6.145321,7.717190,1.663656,-6.150003,10.086035,...,2.862203,1.976548,-0.817164,2.044968,-2.191038,-0.185239,2.933536,-1.361268,5.544809,-2.138781
4,13.451060,-3.473486,-2.162448,-0.605669,9.772358,2.797636,-3.899224,5.122365,-2.800102,-3.232837,...,0.942384,0.280117,-2.641073,1.996963,0.512778,-0.749256,2.476493,3.007772,-0.138860,3.184820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,7.781273,3.490063,-8.716914,10.183494,-1.830992,1.395100,1.099287,0.389329,-2.041018,-1.173125,...,0.933566,0.989863,0.744937,-0.411421,0.592176,0.045836,-1.239901,-1.029403,-0.284587,1.434223
667,3.780461,0.524165,0.702260,2.736404,-1.971068,-0.905211,-1.504362,0.297795,0.437678,-0.884921,...,1.563639,0.018378,0.726326,1.478221,-0.909043,0.576899,0.330480,-0.008074,0.525866,-0.507033
668,3.419803,0.522702,0.271790,-1.523252,-0.762245,0.185145,-0.986453,0.474520,0.527858,0.330443,...,-0.942306,0.515061,0.170787,0.186611,-0.284757,-0.806896,-1.037910,-0.149769,-0.591268,0.827472
669,6.910547,0.362306,-2.079867,4.665526,-1.397047,-1.299842,-1.984956,-0.146718,1.531447,-0.612644,...,1.613700,-0.166640,-0.106522,-0.268259,-0.056615,0.826271,0.312653,-0.248670,-0.000027,0.433370


### The source used scipy 'svds' which is equivalent to 'arpack' algorithm in sklearn, but I am unsure how to implement this methodology in sklearn, so I will demonstrate it with scipy

In [13]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [14]:
sigma = np.diag(sigma)

In [15]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = dfMat.columns)

In [16]:
preds_df

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.054239,0.045130,-0.004835,-0.019817,-0.011284,0.041373,-0.007822,-0.017188,0.012246,0.037670,...,-0.005258,-0.005453,0.012369,-0.004991,-0.004639,-0.019055,0.021402,-0.006365,-0.006098,-0.004819
1,0.419835,1.406440,-0.188807,0.156658,0.268032,0.414698,0.052172,0.044728,-0.020198,2.220256,...,-0.005909,-0.003974,-0.012555,-0.003555,-0.002711,-0.071621,-0.016212,0.001047,-0.001468,-0.006577
2,1.345619,0.266505,-0.011962,0.012278,0.079508,0.090960,-0.122094,0.031327,-0.018023,0.141176,...,-0.002647,-0.002364,-0.010153,0.000277,-0.000116,-0.018063,-0.015761,0.010611,0.006792,-0.006357
3,1.133455,1.046982,0.141275,0.081841,-0.339675,-1.484659,-0.263096,-0.169750,-0.021862,1.611664,...,0.020805,0.000410,0.056040,-0.002817,-0.000767,0.159159,0.087519,-0.030854,-0.021279,0.048529
4,1.389578,1.466495,0.605557,-0.029647,0.729380,-0.118539,-0.026017,0.065577,-0.156655,0.307926,...,-0.007422,-0.011810,0.006644,-0.005159,-0.001249,-0.034658,0.016456,0.001710,-0.004166,-0.001864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,1.445072,0.796437,0.349593,0.118584,0.362366,1.521803,0.537486,-0.001728,0.183798,1.376292,...,-0.004853,-0.006486,-0.001130,-0.001360,0.001742,-0.049968,0.002087,-0.003415,-0.005074,-0.002143
667,0.556714,-0.208781,-0.034636,0.004528,0.010257,0.310522,-0.121078,-0.045900,0.006899,0.004020,...,0.003859,0.001371,-0.002415,0.007073,0.008663,-0.007269,-0.005569,-0.001942,0.000391,0.003827
668,0.728688,-0.135384,0.189810,0.035400,-0.018500,0.242149,0.005227,-0.000577,0.124925,0.030707,...,-0.005106,-0.004248,-0.004701,-0.002367,-0.001945,0.009493,-0.005446,0.001574,-0.000341,-0.007001
669,1.581870,0.084578,-0.046363,-0.013777,0.180043,0.742543,0.004080,-0.009704,-0.016266,0.240628,...,0.005182,0.005733,0.011725,0.006690,0.007145,-0.035748,0.014697,0.007310,0.006698,0.004584


In [62]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how ='left', left_on = 'movieId', right_on = 'id').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[movies_df['id'].isin(user_full['id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'id',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

already_rated, predictions = recommend_movies(preds_df, 3, dfMeta, dfRate, 10)

User 3 has already rated 51 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [67]:
already_rated.loc[:,['userId', 'original_title', 'genres']].head()

,userId,original_title,genres
34,3,License to Wed,"[{'id': 35, 'name': 'Comedy'}]"
5,3,The Million Dollar Hotel,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name..."
7,3,NaN,NaN
37,3,NaN,NaN
17,3,NaN,NaN


In [68]:
predictions.loc[:, ['title','genres']].head()

,title,genres
11,The Million Dollar Hotel,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name..."
12,Terminator 3: Rise of the Machines,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam..."
0,Once Were Warriors,"[{'id': 18, 'name': 'Drama'}]"
9,Solaris,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam..."
20,License to Wed,"[{'id': 35, 'name': 'Comedy'}]"


## Obviously this method has some issues because it has predicted the two movies that have already been rated by the user in this case

This method was a little bit over my head in terms of the math going on in the background and debugging the code from sklearn.